In [1]:
import time
import matplotlib
import matplotlib.pyplot as plt
import numpy as np 
from dsp.Runner import SequenceSolver, distance_based
from dsp.Problem import Problem
from dsp.Solver import DPSolver
from dtspx_ampl import ampl_solve
import pandas as pd
import pprint as pp
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
B1 = '\x1b[1;31m'
B2 = '\x1b[0m'

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# Load Python solutions
# Sequences 0 - 5
exhaustive = np.array([[0], [0, 4, 0], [0, 32, 4, 0], [0, 32, 63, 4, 0], 
                       [0, 32, 30, 63, 4, 0], [0, 8, 5, 30, 63, 4, 0]])
exhaustive_dist = np.array([0, 29.60581726690888, 41.287005153386275, 42.86400500736726, 
                   43.47402889875414, 46.59020901159798])
%store exhaustive
%store exhaustive_dist

# Sequences 0 - 19
trunc_small = np.array([[0], 
               [0, 4, 0], 
               [0, 32, 4, 0], 
               [0, 32, 63, 4, 0], 
               [0, 32, 30, 63, 4, 0], 
               [0, 8, 5, 30, 63, 4, 0], 
               [0, 15, 8, 5, 30, 63, 4, 0],
               [0, 15, 8, 5, 44, 30, 63, 4, 0], 
               [0, 15, 8, 5, 44, 32, 30, 63, 4, 0], 
               [0, 15, 8, 5, 44, 32, 30, 12, 63, 4, 0], 
               [0, 15, 8, 5, 44, 32, 30, 12, 23, 61, 28, 0], 
               [0, 15, 8, 5, 44, 32, 30, 12, 63, 23, 61, 28, 0], 
               [0, 33, 15, 8, 5, 44, 32, 30, 12, 63, 23, 61, 28, 0], 
               [0, 15, 8, 5, 44, 26, 56, 53, 30, 12, 63, 23, 61, 28, 0],  
               [0, 15, 8, 5, 44, 26, 56, 53, 32, 30, 63, 12, 23, 61, 28, 0],
               [0, 15, 8, 5, 44, 26, 56, 53, 32, 30, 4, 63, 12, 23, 61, 28, 0], 
               [0, 15, 8, 5, 44, 26, 56, 53, 32, 30, 38, 12, 63, 4, 23, 61, 28, 0], 
               [0, 15, 8, 5, 44, 26, 56, 53, 32, 30, 4, 63, 23, 12, 43, 7, 61, 28, 0], 
               [0, 15, 8, 5, 44, 26, 56, 53, 32, 30, 4, 63, 12, 43, 7, 16, 23, 61, 28, 0],
               [0, 15, 8, 5, 44, 26, 56, 53, 32, 30, 38, 12, 63, 4, 43, 7, 16, 23, 61, 28, 0]])

trunc_small_dist = np.array([0, 29.60581726690888, 41.287005153386275, 42.86400500736726,
                    43.47402889875414, 46.59020901159798, 47.32008089093999, 48.726690537446615,
                    50.50818642723385, 53.80681170823662, 55.43306651896063, 60.49255776021245,
                    70.10282668607931, 81.35336958118916, 89.47894703426255, 102.34790831028971,
                    123.95825148370193, 161.47257940115142, 182.45969939693046, 206.58742277095322])

%store trunc_small
%store trunc_small_dist

# Sequences 0 - 20
trunc_large = np.array([[0], 
               [0, 4, 0], 
               [0, 32, 4, 0], 
               [0, 32, 63, 4, 0],
               [0, 32, 30, 63, 4, 0], 
               [0, 8, 5, 30, 63, 4, 0],
               [0, 15, 8, 5, 30, 63, 4, 0], 
               [0, 15, 8, 5, 44, 30, 63, 4, 0], 
               [0, 15, 8, 5, 44, 32, 30, 63, 4, 0], 
               [0, 15, 8, 5, 44, 32, 30, 12, 63, 4, 0], 
               [0, 15, 8, 5, 44, 32, 30, 12, 23, 61, 28, 0], 
               [0, 15, 8, 5, 44, 32, 30, 12, 63, 23, 61, 28, 0], 
               [0, 33, 15, 8, 5, 44, 32, 30, 12, 63, 23, 61, 28, 0], 
               [0, 15, 8, 5, 44, 26, 56, 53, 30, 12, 63, 23, 61, 28, 0], 
               [0, 15, 8, 5, 44, 26, 56, 53, 32, 30, 63, 12, 23, 61, 28, 0], 
               [0, 33, 15, 8, 26, 56, 53, 5, 44, 32, 30, 63, 12, 23, 61, 28, 0], 
               [0, 33, 15, 8, 26, 56, 53, 5, 44, 32, 30, 4, 63, 12, 23, 61, 28, 0], 
               [0, 33, 15, 8, 26, 56, 53, 5, 44, 32, 30, 38, 12, 63, 4, 23, 61, 28, 0], 
               [0, 33, 15, 8, 26, 56, 53, 5, 44, 32, 30, 4, 63, 23, 12, 43, 7, 61, 28, 0], 
               [0, 33, 15, 8, 26, 56, 53, 5, 44, 32, 30, 4, 63, 12, 43, 7, 16, 23, 61, 28, 0], 
               [0, 33, 15, 8, 5, 44, 26, 56, 53, 38, 4, 32, 30, 63, 12, 43, 7, 16, 23, 61, 28, 0]])


trunc_large_dist = np.array([0, 29.60581726690888, 41.287005153386275, 42.86400500736726, 43.47402889875414, 
                    46.59020901159798, 47.32008089093999, 48.726690537446615, 50.50818642723385, 
                    53.80681170823662, 55.43306651896063, 60.49255776021245, 70.10282668607931, 
                    81.35336958118916, 89.47894703426255, 99.51380991524988, 110.57946178384833, 
                    135.33922999207147, 169.48623519168441, 190.69125287048908, 217.23216350848233])

%store trunc_large
%store trunc_large_dist

Stored 'exhaustive' (ndarray)
Stored 'exhaustive_dist' (ndarray)
Stored 'trunc_small' (ndarray)
Stored 'trunc_small_dist' (ndarray)
Stored 'trunc_large' (ndarray)
Stored 'trunc_large_dist' (ndarray)


In [5]:
# Solve ampl sequences
ampl_al = []
ampl_seq = []
ampl_sched = []
ampl_dist = []
# Structure i, k, j, l
for al in range(2, 20):
    with open(f"./results/ampl/data_{al}.csv") as fp: 
        df = pd.read_csv(fp)
        df = df.loc[df["z.val"] > 0]
        df = df.iloc[:, 0].str.replace(r"[()]","").str.split(",", expand=True)
        df.columns = ['i', 'k', 'j', 'l']
        
        df = df.apply(pd.to_numeric)
        df.sort_values(by=['k', 'l'], inplace=True)
        
        schedule = df[['k', 'l']].values.ravel()
        _, idx = np.unique(schedule, return_index=True)
        schedule = schedule[np.sort(idx)]
        
        ships = df[['i', 'j']].values.ravel()
        _, idx = np.unique(ships, return_index=True)
        ships = ships[np.sort(idx)]
        ampl_seq.append(ships)
        ampl_sched.append(schedule)
#         print(df)
    with open(f"./results/ampl/sol_{al}.txt") as fp: 
        ampl_dist.append(float(fp.readline()))
    ampl_al.append(al)
ampl_dist = np.array(ampl_dist)

In [11]:
# Plot the pf

x_path = []
y_path = []

# fig = go.FigureWidget(make_subplots(rows=2, cols=1))
fig = go.FigureWidget()


fig.add_trace(go.Scatter(x=np.arange(len(trunc_small_dist)), 
                         y=trunc_small_dist,
                         name="Truncation - 10K",
                         hovertext=[str(_) for _ in trunc_small],
                         mode='markers'))

fig.add_trace(go.Scatter(x=np.arange(len(trunc_large_dist)), 
                         y=trunc_large_dist,
                         hovertext=[str(_) for _ in trunc_large],
                         name="Truncation - 100K"))

fig.add_trace(go.Scatter(x=np.arange(2, 20), 
                         y=ampl_dist,
                         hovertext=[str(_) for _ in ampl_seq],
                         name="AMPL"))

fig.add_trace(go.Scatter(x=np.arange(len(exhaustive)), 
                         y=exhaustive_dist,
                         hovertext=[str(_) for _ in exhaustive],
                         name="Exhaustive"))    


# fig.add_trace(go.Scatter(x=x_path, 
#                          y=y_path,
#                          name="Tour"), row=2, col=1)


# Set options common to all traces with fig.update_traces
fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)
fig.update_layout(title='Evaluated Pareto-Optimal Solutions',
                  yaxis_zeroline=False, xaxis_zeroline=False, 
                  xaxis_title="Sequence Length", yaxis_title="Distance (km)")

# scatter = fig.data[0]
# scatter2 = fig.data[-1]

# # create our callback function
# def update_point(trace, points, selector):
#     S = DPSolver(P, seq=[])
#     sched, dist = S.solve(seq=seq, return_distance=True)
#     x, y = [], []
#     for ship, t in zip(seq, times):
#         # Add first and last times for this ship
#         s = self.problem.get_ship(ship)
#         position = s.get_position(time=t)
#         x.append(position[0])
#         y.append(position[1])

#     scatter2.x = x
#     scatter2.y = y
    
# print(fig.data[0])

fig

FigureWidget({
    'data': [{'hovertext': [[0], [0, 4, 0], [0, 32, 4, 0], [0, 32, 63, 4, 0], [0,
             …

In [54]:
scatter.on_click(update_point)

## Feasibility Checks

In [7]:
T = 6
TW = np.int(np.floor(T/(5/60)+0.1))
np.random.seed(10)
# Data
x_data = np.genfromtxt("data/x.csv", delimiter=",")
y_data = np.genfromtxt("data/y.csv", delimiter=",")

xy_data = np.stack([x_data, y_data], axis=2)

P = Problem(xy_data, T=6)

In [8]:
# number of ships in the working area in time [0, T]
n = xy_data.shape[1]

# Number of time slots in time [0, T]
m = xy_data.shape[0]


first = np.full(n, m, dtype=np.int)  # First slot when ship i is in the work area
last = np.full(n, 0, dtype=np.int)  # Last slot when ship i is in the work area

shipn = np.full(n, np.nan, dtype=np.int)  # First slot when ship i is in the work area
shipo = np.full(n, np.nan, dtype=np.int)  # Last slot when ship i is in the work area


# For loop to fill out first, and last array
for s in range(n):
    c = np.all(xy_data[:TW+1, s] != 0, axis=1)
    non_zero = np.where(c)[0]
    if len(non_zero):
        first[s] = non_zero[0]
        last[s] = non_zero[-1]
        


# x = np.concatenate((first[:, None], last[:, None]), axis=1)
r = -1
for j in range(0, n):
    if first[j] <= last[j]:
        r += 1   
        shipn[r] = j
        shipo[j] = r
# x = np.concatenate((shipo[:, None], shipn[:, None]), axis=1)
for i, s in enumerate(ampl_seq):
    print(f"Sequence Length {i+2}")
#     print("AMPL Modified Sequence: ", s)
    new_s = []
    s = s.astype(int)-1
    for _s in s: 
        if _s < 0 or _s > n:
            continue
        new_s.append(shipn[_s])
    s = new_s
    print("AMPL Sequence: ", s)
    S = DPSolver(P, seq=[])
    if s[0] != 0: 
        s = np.append([0], s)
    if s[-1] != 0: 
        s = np.append(s, [0])
        
    res = S.solve(seq=s, return_distance=True)
    if res: 
        sched, dist = res
        if len(sched) == len(ampl_sched[i]):
            difference = [_i for _i, _e in enumerate(sched) if _e != ampl_sched[i][_i]]
        else: difference = []
        print(f"Computed Schedule: {' -> '.join(B1 + str(j) + B2 if difference and _i in difference else str(j) for _i, j in enumerate(sched))}")
        print(f"AMPL Schedule:     {' -> '.join(B1 + str(int(j)) + B2 if difference and _i in difference else str(int(j)) for _i, j in enumerate(ampl_sched[i]))}")
        print(f"AMPL Computed Distance: {B1}{dist:.4f} km {B2}")
        print(f"AMPL Distance:          {B1}{ampl_dist[i]:.4f} km {B2}")
    
    print()


Sequence Length 2
AMPL Sequence:  [32, 4]
Computed Schedule: 0 -> 16 -> 51 -> 72
AMPL Schedule:     0 -> 16 -> 51 -> 72
AMPL Computed Distance: 41.2870 km 
AMPL Distance:          41.2870 km 

Sequence Length 3
AMPL Sequence:  [32, 63, 4]
Computed Schedule: 0 -> 16 -> 36 -> 51 -> 72
AMPL Schedule:     0 -> 16 -> 36 -> 51 -> 72
AMPL Computed Distance: 42.8640 km 
AMPL Distance:          42.8640 km 

Sequence Length 4
AMPL Sequence:  [32, 30, 63, 4]
Computed Schedule: 0 -> 16 -> 24 -> 36 -> 51 -> 72
AMPL Schedule:     0 -> 16 -> 24 -> 36 -> 51 -> 72
AMPL Computed Distance: 43.4740 km 
AMPL Distance:          43.4740 km 

Sequence Length 5
AMPL Sequence:  [8, 5, 30, 63, 4]
Computed Schedule: 0 -> 9 -> 12 -> 24 -> 36 -> 51 -> 72
AMPL Schedule:     0 -> 9 -> 12 -> 24 -> 36 -> 51 -> 72
AMPL Computed Distance: 46.5902 km 
AMPL Distance:          46.5902 km 

Sequence Length 6
AMPL Sequence:  [15, 8, 5, 30, 63, 4]
Computed Schedule: 0 -> 8 -> 9 -> 12 -> 24 -> 36 -> 51 -> 72
AMPL Schedule:     

In [10]:
S = DPSolver(P, seq=[])
print(S.solve(seq=[0,33, 15, 26, 8, 56, 53, 38, 5, 44, 32, 30, 63, 4, 12, 23, 61, 28, 0], return_distance=True))
# print(ampl_sched)


None
